In [1]:
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
df=pd.read_csv("Superstore_clean_second_python.csv")

In [3]:
df.head()

,order_date,order_month,order_year,ship_date,ship_mode,customer_id,segment,city,state,region,product_id,category,subcategory,sales,quantity,discount,profit
0,2016-11-08,November,2016,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,2016-11-08,November,2016,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,731.9400,3,0.00,219.5820
2,2016-06-12,June,2016,2016-06-16,Second Class,DV-13045,Corporate,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,2015-10-11,October,2015,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,957.5775,5,0.45,-383.0310
4,2015-10-11,October,2015,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,2,0.20,2.5164


In [18]:
total_sales = float(round(df['sales'].sum(),2))

In [19]:
total_sales

2297200.86

In [16]:
total_profit = float(round(df['profit'].sum(), 2))

In [17]:
total_profit

286397.02

In [21]:
profit_margin = float(round(total_profit / total_sales * 100,2))

In [22]:
profit_margin

12.47

In [23]:
num_loss_orders = df[df['profit'] < 0].shape[0]

In [33]:
num_loss_orders = df[df['profit'] < 0].shape[0]
total_orders = df.shape[0]
percent_loss_orders = float(round(num_loss_orders / total_orders * 100,2))

In [34]:
percent_loss_orders

18.72

In [35]:
kpi = {
    "total_sales": total_sales,
    "total_profit": total_profit,
    "profit_margin": profit_margin,
    "loss_orders_pct": percent_loss_orders
}

In [36]:
monthly_summary = df.groupby(['order_year', 'order_month']).agg(
    total_sales=('sales', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

In [37]:
monthly_summary_json = (
    monthly_summary
    .assign(
        month=lambda x: x['order_month'].astype(str).str.zfill(2),
        year=lambda x: x['order_year'].astype(str)
    )
    .assign(
        period=lambda x: x['year'] + "-" + x['month']
    )
    [["period", "total_sales", "total_profit"]]
    .to_dict(orient="records")
)

In [39]:
df['profit_margin'] = df['profit'] / df['sales']

In [40]:
category_summary = (
    df.groupby(['category', 'subcategory'])
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_date', 'nunique')  # унікальні замовлення по даті
      )
      .reset_index()
)

In [53]:
category_summary_json = (
    category_summary
    .assign(
        total_sales=lambda x: x['total_sales'].round(2),
        total_profit=lambda x: x['total_profit'].round(2),
        avg_profit_margin=lambda x: x['avg_profit_margin'].round(2)
    )
    .to_dict(orient="records")
)

In [47]:
region_summary = (
    df.groupby('region')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_date', 'nunique')
      )
      .reset_index()
)

In [54]:
region_summary_json = (
    region_summary
    .round({
        "total_sales": 2,
        "total_profit": 2,
        "profit_margin": 2
    })
    .to_dict(orient="records")
)

In [50]:
segment_summary = (
    df.groupby('segment')
      .agg(
          total_sales=('sales', 'sum'),
          total_profit=('profit', 'sum'),
          avg_profit_margin=('profit_margin', 'mean'),
          orders_count=('order_date', 'nunique')
      )
      .reset_index()
)

In [55]:
segment_summary_json = (
    segment_summary
    .round({
        "total_sales": 2,
        "total_profit": 2,
        "avg_profit_margin": 2
    })
    .to_dict(orient="records")
)

In [56]:
category_summary.sort_values('total_profit', ascending=False)

,category,subcategory,total_sales,total_profit,avg_profit_margin,orders_count
14,Technology,Copiers,149528.0300,55617.8249,0.317194,67
16,Technology,Phones,330007.0540,44515.7306,0.119222,572
13,Technology,Accessories,167380.3180,41936.6357,0.218210,512
10,Office Supplies,Paper,78479.2060,34053.5693,0.425600,730
6,Office Supplies,Binders,203412.7330,30221.7633,-0.199595,776
1,Furniture,Chairs,328449.1030,26590.1663,0.043900,416
11,Office Supplies,Storage,223843.6080,21278.8264,0.089113,558
4,Office Supplies,Appliances,107532.1610,18138.0054,-0.156869,366
2,Furniture,Furnishings,91705.1640,13059.1436,0.137066,599
7,Office Supplies,Envelopes,16476.4020,6964.1767,0.423140,218


In [57]:
region_summary.sort_values('total_profit', ascending=False)

,region,total_sales,total_profit,avg_profit_margin,orders_count
3,West,725457.8245,108418.4489,0.219487,845
1,East,678781.2400,91522.7800,0.167227,787
2,South,391721.9050,46749.4303,0.163519,604
0,Central,501239.8908,39706.3625,-0.104073,720


In [58]:
segment_summary.sort_values('total_profit', ascending=False)

,segment,total_sales,total_profit,avg_profit_margin,orders_count
0,Consumer,1.161401e+06,134119.2092,0.112050,1044
1,Corporate,7.061464e+05,91979.1340,0.121203,830
2,Home Office,4.296531e+05,60298.6785,0.142870,610


In [59]:
report_input = {
    "kpi": kpi,
    "monthly_summary": monthly_summary_json,
    "category_summary": category_summary_json,
    "region_summary": region_summary_json,
    "segment_summary": segment_summary_json
}

In [60]:
report_input_json = json.dumps(report_input, indent=2)

In [61]:
print(report_input_json)

{
  "kpi": {
    "total_sales": 2297200.86,
    "total_profit": 286397.02,
    "profit_margin": 12.47,
    "loss_orders_pct": 18.72
  },
  "monthly_summary": [
    {
      "period": "2014-April",
      "total_sales": 28295.345,
      "total_profit": 3488.8352
    },
    {
      "period": "2014-August",
      "total_sales": 27909.4685,
      "total_profit": 5318.105
    },
    {
      "period": "2014-December",
      "total_sales": 69545.6205,
      "total_profit": 8983.5699
    },
    {
      "period": "2014-February",
      "total_sales": 4519.892,
      "total_profit": 862.3084
    },
    {
      "period": "2014-January",
      "total_sales": 14236.895,
      "total_profit": 2450.1907
    },
    {
      "period": "2014-July",
      "total_sales": 33946.393,
      "total_profit": -841.4826000000002
    },
    {
      "period": "2014-June",
      "total_sales": 34595.1276,
      "total_profit": 4976.5244
    },
    {
      "period": "2014-March",
      "total_sales": 55691.009,
      "

In [62]:
load_dotenv()

True

In [63]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [64]:
PRODUCTION_PROMPT_V2 = """
You are a Senior Business Analyst preparing an executive-level business performance report for management.

Rules:
- Use ONLY the provided input data.
- You MAY perform basic aggregations, ratios, and consistency checks strictly based on the data.
- Do NOT invent data, assumptions, or benchmarks.
- Ensure all financial metrics are logically consistent (e.g., profit cannot exceed sales).
- If inconsistencies or data quality issues are detected, explicitly flag them as business risks.
- Do NOT mention JSON, Python, APIs, models, or technical implementation details.
- Write clearly, concisely, and decisively for executives.

Your goal is NOT to describe the data, but to support business decisions.

Required Output Format (Markdown only):

# Business Performance Report

## Management Summary
Provide a concise executive overview that answers:
- What is happening to the business overall
- Where the business is losing money or underperforming
- Why this matters financially
- What management should prioritize next

## Executive Summary
Include only decision-critical metrics:
- Total Sales
- Total Profit
- Overall Profit Margin (%)
- Share of loss-making orders (%)
- Key financial risk or opportunity

## Key Business Insights
Provide 4–6 insights. EACH insight must follow this structure:

Insight: What is happening (non-obvious finding)
Why it matters: Financial or operational implication
Business impact: Risk or opportunity for management

Avoid descriptive statements such as “X performs well”.

## Recommendations
Provide 3–5 prioritized, action-oriented recommendations.

Each recommendation must:
- Target a specific category, region, or segment
- Address a quantified issue identified above
- Indicate expected business effect (directional, not speculative)

## Key Metrics
Present a concise executive table or bullet list with:
- Sales
- Profit
- Profit Margin (%)
- Loss Orders (%)

## Data Limitations & Risks
Explicitly state:
- Missing fields or unavailable dimensions
- Data quality issues
- Areas requiring further validation

Return ONLY the final report in Markdown.
"""

In [65]:
prompt = f"""
{PRODUCTION_PROMPT_V2}

INPUT DATA:
{report_input_json}
"""


In [66]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert business analyst."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

In [67]:
business_report_md = response.choices[0].message.content

In [71]:
print(business_report_md.replace("\\n", "\n"))

# Business Performance Report

## Management Summary
The business is currently experiencing a total sales figure of approximately $2.3 million, with a profit of around $286,000, resulting in a profit margin of 12.47%. However, 18.72% of orders are loss-making, indicating significant areas of underperformance. The categories of Furniture and certain segments within Office Supplies are particularly problematic, with negative profit margins impacting overall profitability. This situation is financially critical as it undermines the sustainability of operations and could lead to cash flow issues if not addressed. Management should prioritize strategies to reduce loss-making orders and enhance profitability in underperforming categories.

## Executive Summary
- **Total Sales:** $2,297,200.86
- **Total Profit:** $286,397.02
- **Overall Profit Margin:** 12.47%
- **Share of Loss-Making Orders:** 18.72%
- **Key Financial Risk:** High percentage of loss-making orders, particularly in Furniture a

In [72]:
with open("business_report.md", "w", encoding="utf-8") as f:
    f.write(business_report_md)